<a href="https://colab.research.google.com/github/ipvpc/Cloudstack-Tools/blob/master/examples/options-trading-basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [3]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

api_key = "PKURKFASXB04JMSMLTGX"
secret_key = "noh5VrukdfEzlPfRczkJ6ukU5tIMc5jG4eNhOP8c"


#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [5]:
# install alpaca-py
! python3 -m pip install alpaca-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 8.8 MB/s eta 0:00:00


In [6]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [7]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [8]:
# check version of alpaca-py
alpaca.__version__

'0.32.0'

# Trading Client

In [9]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [10]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3YTMOTNZFN',
    'accrued_fees': '0',
    'buying_power': '199547.24',
    'cash': '99773.62',
    'created_at': datetime.datetime(2024, 10, 18, 15, 21, 37, 460741, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '99773.62',
    'id': UUID('2c0beb25-d961-4e5a-9e3f-c9c60284c3b0'),
    'initial_margin': '0',
    'last_equity': '99773.62',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99773.62',
    'options_approved_level': 2,
    'options_buying_power': '99773.62',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '99773.62',
    'regt_buying_power': '199547.24',
    'short_market_value': '0',
    

In [11]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [12]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('67b56554-dd21-4010-99f8-f256e7ab5db1'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'EBET INC Common Shares',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'EBET',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('e2b309ba-869b-4405-91f6-565db136fdc4'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'Independence Contract

In [13]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '218.34',
                                'close_price_date': datetime.date(2024, 11, 11),
                                'expiration_date': datetime.date(2024, 11, 15),
                                'id': 'fc7aa684-bdc4-405b-93bb-b095c58c05a3',
                                'name': 'AAPL Nov 15 2024 5 Call',
                                'open_interest': '1',
                                'open_interest_date': datetime.date(2024, 11, 11),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 5.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241115C00005000',
                                'tradable': True,
                                'type': <Contr

In [14]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': '212.95',
                                'close_price_date': datetime.date(2024, 10, 10),
                                'expiration_date': datetime.date(2024, 11, 15),
                                'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
                                'name': 'AAPL Nov 15 2024 15 Call',
                                'open_interest': None,
                                'open_interest_date': None,
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 15.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL241115C00015000',
                                'tradable': True,
                                'type': <ContractType.CALL: 'call'

In [15]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': '212.95',
    'close_price_date': datetime.date(2024, 10, 10),
    'expiration_date': datetime.date(2024, 11, 15),
    'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
    'name': 'AAPL Nov 15 2024 15 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 15.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241115C00015000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [16]:
# get options contract by id
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': '212.95',
    'close_price_date': datetime.date(2024, 10, 10),
    'expiration_date': datetime.date(2024, 11, 15),
    'id': 'a3cbcd4c-aeb6-40e1-b871-3cbf18da88cf',
    'name': 'AAPL Nov 15 2024 15 Call',
    'open_interest': None,
    'open_interest_date': None,
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 15.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL241115C00015000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [17]:
# get put options contracts
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:2]

[{   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 14),
     'id': '057a0b5e-e262-4e9e-8f55-4cda1851ad87',
     'name': 'SPY Nov 14 2024 300 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 300.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY241114P00300000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 11, 14),
     'id': 'a38d8654-6379-4c0e-b307-08089bcbb148',
     'name': 'SPY Nov 14 2024 305 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,


In [18]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.02',
    'close_price_date': datetime.date(2024, 11, 12),
    'expiration_date': datetime.date(2024, 11, 14),
    'id': '18acb8f5-df91-4a53-ab4f-eb13a8090644',
    'name': 'SPY Nov 14 2024 520 Put',
    'open_interest': '1919',
    'open_interest_date': datetime.date(2024, 11, 11),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 520.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY241114P00520000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [ ]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 1,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

In [ ]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

In [17]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[]

In [ ]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


In [ ]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

In [ ]:
# close the option position
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "1")
)

In [ ]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [18]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

keyboard interrupt, bye


# Market Data (Historical)

In [19]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [20]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

open  high   low  close  volume  \
symbol             timestamp                                                    
SPY241114P00520000 2024-11-12 14:00:00+00:00  0.01  0.01  0.01   0.01    15.0   
                   2024-11-12 15:00:00+00:00  0.02  0.02  0.01   0.01    80.0   

                                              trade_count      vwap  
symbol             timestamp                                         
SPY241114P00520000 2024-11-12 14:00:00+00:00          2.0  0.010000  
                   2024-11-12 15:00:00+00:00          6.0  0.015625

In [21]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

exchange  price  size  \
symbol             timestamp                                                
SPY241114P00520000 2024-11-12 14:58:09.095879+00:00        J   0.01   1.0   
                   2024-11-12 14:58:09.095940+00:00        B   0.01  14.0   

                                                    conditions  
symbol             timestamp                                    
SPY241114P00520000 2024-11-12 14:58:09.095879+00:00          I  
                   2024-11-12 14:58:09.095940+00:00          I

In [22]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

{'A': 'AMEX - NYSE American',
 'B': 'BOX - Boston Options Exchange',
 'C': 'CBOE - Cboe Options Exchange',
 'D': 'EMERALD - Miami International Stock Exchange Emerald Options',
 'E': 'EDGX - Cboe EDGX Options Exchange',
 'H': 'GEMX - Nasdaq GEMX',
 'I': 'ISE - Nasdaq International Securities Exchange',
 'J': 'MRX - Nasdaq MRX',
 'M': 'MIAX - Miami International Stock Exchange',
 'N': 'NYSE - NYSE Arca',
 'O': 'OPRA - Options Price Reporting Authority',
 'P': 'PEARL - Miami International Stock Exchange Pearl Options',
 'Q': 'NASD - Nasdaq Options',
 'S': 'SPHR - Miami International Stock Exchange Sapphire Options',
 'T': 'BX - Nasdaq BX Options',
 'U': 'MEMX - Members Options Exchange',
 'W': 'C2 - Cboe C2 Options Exchange',
 'X': 'PHLX - Nasdaq PHLX',
 'Z': 'BATS - Cboe BZX Options Exchange'}

In [23]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

{'SPY241114P00520000': {   'ask_exchange': 'E',
     'ask_price': 0.01,
     'ask_size': 1548.0,
     'bid_exchange': '?',
     'bid_price': 0.0,
     'bid_size': 0.0,
     'conditions': 'A',
     'symbol': 'SPY241114P00520000',
     'tape': None,
     'timestamp': datetime.datetime(2024, 11, 13, 21, 14, 59, 470075, tzinfo=TzInfo(UTC))}}

In [24]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

{'SPY241114P00520000': {   'conditions': 'j',
     'exchange': 'D',
     'id': None,
     'price': 0.02,
     'size': 1.0,
     'symbol': 'SPY241114P00520000',
     'tape': None,
     'timestamp': datetime.datetime(2024, 11, 12, 21, 12, 23, 818924, tzinfo=TzInfo(UTC))}}

In [25]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

{'SPY241114P00520000': {   'greeks': None,
     'implied_volatility': None,
     'latest_quote': {   'ask_exchange': 'E',
                         'ask_price': 0.01,
                         'ask_size': 1548.0,
                         'bid_exchange': '?',
                         'bid_price': 0.0,
                         'bid_size': 0.0,
                         'conditions': 'A',
                         'symbol': 'SPY241114P00520000',
                         'tape': None,
                         'timestamp': datetime.datetime(2024, 11, 13, 21, 14, 59, 470075, tzinfo=TzInfo(UTC))},
     'latest_trade': {   'conditions': 'j',
                         'exchange': 'D',
                         'id': None,
                         'price': 0.02,
                         'size': 1.0,
                         'symbol': 'SPY241114P00520000',
                         'tape': None,
                         'timestamp': datetime.datetime(2024, 11, 12, 21, 12, 23, 818924, tzinfo=TzInfo(UTC))

In [26]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

{'SPY241113C00494000': {   'greeks': None,
     'implied_volatility': None,
     'latest_quote': {   'ask_exchange': 'N',
                         'ask_price': 104.19,
                         'ask_size': 73.0,
                         'bid_exchange': 'N',
                         'bid_price': 103.02,
                         'bid_size': 73.0,
                         'conditions': 'A',
                         'symbol': 'SPY241113C00494000',
                         'tape': None,
                         'timestamp': datetime.datetime(2024, 11, 13, 21, 14, 57, 333728, tzinfo=TzInfo(UTC))},
     'latest_trade': None,
     'symbol': 'SPY241113C00494000'},
 'SPY241113C00562000': {   'greeks': None,
     'implied_volatility': None,
     'latest_quote': {   'ask_exchange': 'N',
                         'ask_price': 36.19,
                         'ask_size': 92.0,
                         'bid_exchange': 'N',
                         'bid_price': 35.02,
                         'bid_size':

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols)
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()